In [1]:
from lib.yolo_inference import yolo_infer2
from lib.yolo_inference import prepare_yolo
from lib.yolo_inference import get_bbox
from lib.yolo_inference import infer
from lib.emotic import Emotic
from PIL import Image
import cv2
import numpy as np
import time
import torch
import os
from http.server import BaseHTTPRequestHandler, HTTPServer

In [2]:
hostName = "localhost"
serverPort = 9999

In [3]:
cam = cv2.VideoCapture(0) #0=front-cam, 1=back-cam
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1300)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 1500)

True

In [4]:
class MyServer(BaseHTTPRequestHandler):
    def do_GET(self):
        emotion = detectEmotion()
        self.send_response(200)
        self.send_header("Content-type", "text/html")
        self.end_headers()
        self.wfile.write(bytes("<html><head><title>https://pythonbasics.org</title></head>", "utf-8"))
        #self.wfile.write(bytes("<p>Request: %s</p>" % self.path, "utf-8"))
        self.wfile.write(bytes("<body>", "utf-8"))
        self.wfile.write(bytes("<p>"+emotion+"</p>", "utf-8"))
        self.wfile.write(bytes("</body></html>", "utf-8"))
        #self.wfile.write(bytes("annoyed"))

In [5]:
cat = ['Affection', 'Anger', 'Annoyance', 'Anticipation', 'Aversion', 'Confidence', 'Disapproval', 'Disconnection', \
          'Disquietment', 'Doubt/Confusion', 'Embarrassment', 'Engagement', 'Esteem', 'Excitement', 'Fatigue', 'Fear','Happiness', \
          'Pain', 'Peace', 'Pleasure', 'Sadness', 'Sensitivity', 'Suffering', 'Surprise', 'Sympathy', 'Yearning']
cat2ind = {}
ind2cat = {}
for idx, emotion in enumerate(cat):
    cat2ind[emotion] = idx
    ind2cat[idx] = emotion

vad = ['Valence', 'Arousal', 'Dominance']
ind2vad = {}
for idx, continuous in enumerate(vad):
    ind2vad[idx] = continuous

In [6]:
# we need to normalise the demo image using the statistics of the training data the model was trained on
context_mean = [0.4690646, 0.4407227, 0.40508908]
context_std = [0.2514227, 0.24312855, 0.24266963]
body_mean = [0.43832874, 0.3964344, 0.3706214]
body_std = [0.24784276, 0.23621225, 0.2323653]
context_norm = [context_mean, context_std]
body_norm = [body_mean, body_std]


In [7]:
model_path = 'experiment/models/'
result_path = 'experiment/results/'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
yolo = prepare_yolo(model_path)
yolo = yolo.to(device)
yolo.eval()

thresholds = torch.FloatTensor(np.load(os.path.join(result_path, 'val_thresholds.npy'))).to(device) 
model_context = torch.load(os.path.join(model_path,'model_context1.pth')).to(device)
model_body = torch.load(os.path.join(model_path,'model_body1.pth')).to(device)
emotic_model = torch.load(os.path.join(model_path,'model_emotic1.pth')).to(device)
models = [model_context, model_body, emotic_model]
  
result_file_path = None
return_image = None



prepared yolo model


In [8]:
def detectEmotion():
    detectedEmo = "" 
    ret, img = cam.read()    ## predict yolo
    image_context = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    emo_vals = {}
    try:
        bbox_yolo = get_bbox(yolo, device, image_context)
        for ipbx, pred_bbox in enumerate(bbox_yolo):
            emo_vals[f"bbox_{ipbx}"] = {'cont': [], 'cat': None}
            pred_cat, pred_cont = infer(context_norm, body_norm, ind2cat, ind2vad, device, thresholds, models, image_context=image_context, bbox=pred_bbox, to_print=False)
            '''write_text_vad = list()
            for continuous in pred_cont:
                write_text_vad.append(str('%.1f' %(continuous)))
                emo_vals[f"bbox_{ipbx}"]['cont'].append(continuous)
            emo_vals[f"bbox_{ipbx}"]['cat'] = pred_cat
            write_text_vad = 'vad ' + ' '.join(write_text_vad) 
            image_context = cv2.rectangle(image_context, (pred_bbox[0], pred_bbox[1]),(pred_bbox[2] , pred_bbox[3]), (255, 0, 0), 3)
            cv2.putText(image_context, write_text_vad, (pred_bbox[0], pred_bbox[1] - 5), cv2.FONT_HERSHEY_PLAIN, 1, (0,139,139), 2)
            for i, emotion in enumerate(pred_cat):
                cv2.putText(image_context, emotion, (pred_bbox[0], pred_bbox[1] + (i+1)*12), cv2.FONT_HERSHEY_PLAIN, 1, (0,139,139), 2)'''
            detectedEmo = pred_cat[0]
    except Exception as e:
        #print ('Exception for image ',image_path)
        print (e)


    #return_image = Image.fromarray(image_context)
            ## press q or Esc to quit 
    #print(xs)
    #print(type(r))
    #out.write(np.array(return_image))
    #opencvImage = cv2.cvtColor(np.array(return_image), cv2.COLOR_RGB2BGR)

    #cv2.imshow("", opencvImage)'''
    #time.sleep(2)
    #if (cv2.waitKey(1) & 0xFF == ord("q")) or (cv2.waitKey(1)==27):
        #break## close camera
    #cam.release()
    #cv2.destroyAllWindows()
    return detectedEmo

In [ ]:
webServer = HTTPServer((hostName, serverPort), MyServer)
print("Server started http://%s:%s" % (hostName, serverPort))

try:
    webServer.serve_forever()
except KeyboardInterrupt:
    pass

webServer.server_close()
cam.release()
print("Server stopped.")

Server started http://localhost:9999
'NoneType' object is not subscriptable


127.0.0.1 - - [30/Sep/2021 08:24:55] "GET /detect HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2021 08:26:18] "GET /detect HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2021 08:26:22] "GET /detect HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2021 08:26:26] "GET /detect HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2021 08:26:31] "GET /detect HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2021 08:26:36] "GET /detect HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2021 08:26:44] "GET /detect HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2021 08:26:48] "GET /detect HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2021 08:26:53] "GET /detect HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2021 08:26:57] "GET /detect HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2021 08:27:01] "GET /detect HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2021 08:27:06] "GET /detect HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2021 08:27:10] "GET /detect HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2021 08:33:33] "GET /detect HTTP/1.1" 200 -
127.0.0.1 - - [30/Sep/2021 08:33:37] "GET /detect HTTP/1.1" 200 -
127.0.0.1 

(False, None)